## Clone the Repository

In [ ]:
!git clone https://github.com/giumanuz/fastercnn-pytorch-training-pipeline.git

We will execute all the code within the cloned project directory, that is `fastercnn-pytorch-training-pipeline`.

In [ ]:
import torch
torch.__version__

In [ ]:
# Enter the repo directory.
%cd fastercnn-pytorch-training-pipeline/

In [ ]:
%%writefile requirements.txt
# Base-------------------------------------
albumentations==1.1.0
ipython
jupyter
matplotlib
opencv-python>=4.1.1.26
opencv-python-headless>=4.1.1.26
Pillow
PyYAML
scikit-image
scikit-learn
scipy
# torch==2.0.0
# torchvision==0.15.1
numpy
protobuf<=3.20.1
pandas
tqdm

# Logging----------------------------------
wandb
tensorboard

# Model summary----------------------------
torchinfo

# Extras-----------------------------------
pycocotools>=2.0.2
setuptools==59.5.0
torchmetrics # Evaluation

# Transformer based models.
vision_transformers

# Export-----------------------------------
# onnxruntime
# onnx
# onnxruntime-gpu

In [ ]:
# Install the Requirements
!pip install -r requirements.txt

## Create the Custom Dataset YAML File

The YAML file should contain:
* `TRAIN_DIR_IMAGES`: Path to the training images directory.
* `TRAIN_DIR_LABELS`: Path to the training labels directory containing the XML files. Can be the same as `TRAIN_DIR_IMAGES`.
* `VALID_DIR_IMAGES`: Path to the validation images directory.
* `VALID_DIR_LABELS`: Path to the validation labels directory containing the XML files. Can be the same as `VALID_DIR_IMAGES`.
* `CLASSES`: All the class names in the dataset along with the `__background__` class as the first class.
* `NC`: The number of classes. This should be the number of classes in the dataset + the background class. If the number of classes in the dataset are 7, then `NC` should be 8.
* `SAVE_VALID_PREDICTION_IMAGES`: Whether to save the prediction results from the validation loop or not.

In [ ]:
%%writefile data_configs/custom_data.yaml
# Images and labels direcotry should be relative to train.py
TRAIN_DIR_IMAGES: '../split_photos/train'
TRAIN_DIR_LABELS: '../split_photos/train'
VALID_DIR_IMAGES: '../split_photos/val'
VALID_DIR_LABELS: '../split_photos/val'

# Class names.
CLASSES: [
    '__background__',
    'bad_weld', 'good_weld'
]

# Number of classes (object classes + 1 for background class in Faster RCNN).
NC: 3

# Whether to save the predictions of the validation set while training.
SAVE_VALID_PREDICTION_IMAGES: True

## Training

For this training example we use:
* The official Faster RCNN ResNet50 FPN model.
* Batch size of 8. You may change it according to the GPU memory available.

In [ ]:
!wandb disabled

In [ ]:
!python train.py --data data_configs/custom_data.yaml --epochs 300 --model fasterrcnn_resnet50_fpn_v2 --name custom_training --batch 8 --lr=1e-4 --imgsz=500  --patience=10

## Visualize Validation Results

Check out a few validation results from `outputs/training/custom_training` directory.

In [ ]:
import matplotlib.pyplot as plt
import glob as glob

In [ ]:
results_dir_path = 'outputs/training/custom_training'
valid_images = glob.glob(f"{results_dir_path}/*.jpg")

for i in range(3):
    plt.figure(figsize=(10, 7))
    image = plt.imread(valid_images[i])
    plt.imshow(image)
    plt.axis('off')
    plt.show()

## Evaluation

In [ ]:
# Verbose mAP.
!python eval.py --weights outputs/training/custom_training/best_model.pth --data data_configs/custom_data.yaml --model fasterrcnn_resnet50_fpn_v2 --verbose